In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
import json
from selenium import webdriver
from selenium.common.exceptions import WebDriverException

In [33]:
countries = pd.read_csv('countries.csv')['code'].tolist()

df = pd.read_csv('hs_info - Copy.csv', dtype={'id': str, 'code' : str,'parent_id': str})
df["code"] = df["code"].fillna("").astype(str)
df["code"] = df["code"].apply(lambda x: x.zfill(len(x) + 1) if len(x) % 2 == 1 and x else x)

# df['tariff_crawled'] = ''
df

C:\Users\nguye\AppData\Local\Temp\ipykernel_40872\456977068.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('hs_info - Copy.csv', dtype={'id': str, 'code' : str,'parent_id': str})


,id,code,description,hasChildren,section,index,crawled,country,parent_id,error_message,selectable,intervalMin,intervalMax,tariff_crawled
0,-1,,Live animals; animal products,True,"{'description': 'SECTION I', 'chapterFrom': '0...",0.0,Done,AT,NaN,NaN,NaN,NaN,NaN,NaN
1,-2,,Vegetable products,True,"{'description': 'SECTION II', 'chapterFrom': '...",1.0,Done,AT,NaN,NaN,NaN,NaN,NaN,NaN
2,-3,,Animal or vegetable fats and oils and their cl...,True,"{'description': 'SECTION III', 'chapterFrom': ...",2.0,Done,AT,NaN,NaN,NaN,NaN,NaN,NaN
3,-4,,"Prepared foodstuffs; beverages, spirits and vi...",True,"{'description': 'SECTION IV', 'chapterFrom': '...",3.0,Done,AT,NaN,NaN,NaN,NaN,NaN,NaN
4,-5,,Mineral products,True,"{'description': 'SECTION V', 'chapterFrom': '2...",4.0,Done,AT,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90126,21268,6804210020,Discs<br> <br>- of synthetic diamonds which ar...,False,NaN,0.0,NaN,HR,18368,NaN,True,NaN,NaN,NaN
90127,21368,6804210030,Steel wire used for cropping and squaring semi...,False,NaN,1.0,NaN,HR,18368,NaN,True,NaN,NaN,NaN
90128,21468,6804210090,Other,False,NaN,2.0,NaN,HR,18368,NaN,True,NaN,NaN,NaN
90129,21668,,"Of artificial abrasives, with binder",True,NaN,0.0,NaN,HR,10268,NaN,NaN,6.804221e+09,6.804225e+09,NaN


In [34]:
df.columns

Index(['id', 'code', 'description', 'hasChildren', 'section', 'index',
       'crawled', 'country', 'parent_id', 'error_message', 'selectable',
       'intervalMin', 'intervalMax', 'tariff_crawled'],
      dtype='object')

In [35]:
def fetch_tariff_info_with_retry(code,country,retries=5, delay=5):
    """Gửi request với retry khi lỗi."""
    for attempt in range(retries):
        try:
            url = f"https://trade.ec.europa.eu/access-to-markets/api/tariffs/get/{code}/VN/{country}"
            response = requests.get(url, timeout=60)
            response.encoding = 'utf-8'
            if response.status_code == 200:
                return response
        except requests.RequestException as e:
            print(f"Request error: {e} - Retrying {attempt+1}/{retries}...")
            time.sleep(delay)
    return None

In [36]:
filtered_df = df.loc[(df['country'] == 'AT') & (df['selectable'].notna()) & ((df['tariff_crawled'] == '') | pd.isna(df['tariff_crawled']) | (df['tariff_crawled'] == 'Error'))]


In [37]:
filtered_df

,id,code,description,hasChildren,section,index,crawled,country,parent_id,error_message,selectable,intervalMin,intervalMax,tariff_crawled
65179,405,0501,"Human hair, unworked, whether or not washed or...",False,NaN,0.0,NaN,AT,305,NaN,True,NaN,NaN,NaN
65186,2105,0510,"Ambergris, castoreum, civet and musk; canthari...",False,NaN,7.0,NaN,AT,305,NaN,True,NaN,NaN,NaN
65219,1708,0814,Peel of citrus fruit or melons (including wate...,False,NaN,13.0,NaN,AT,308,NaN,True,NaN,NaN,NaN
65222,609,0903,Maté,False,NaN,2.0,NaN,AT,309,NaN,True,NaN,NaN,NaN
65246,1311,1109,"Wheat gluten, whether or not dried",False,NaN,8.0,NaN,AT,311,NaN,True,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90103,407220,2007993947,Other,False,NaN,1.0,NaN,AT,406520,NaN,True,NaN,NaN,NaN
90104,407320,2007993951,Containing less than 70 % by weight of sugar,False,NaN,0.0,NaN,AT,406720,NaN,True,NaN,NaN,NaN
90105,407420,2007993952,Other,False,NaN,1.0,NaN,AT,406720,NaN,True,NaN,NaN,NaN
90106,407520,2007993954,Containing less than 70 % by weight of sugar,False,NaN,0.0,NaN,AT,406820,NaN,True,NaN,NaN,NaN


In [38]:
fetch_tariff_info_with_retry

<function __main__.fetch_tariff_info_with_retry(code, country, retries=5, delay=5)>

In [39]:
for country in countries:
    while ((df['country'] == country) & (df['selectable'].notna()) & ((df['tariff_crawled'] == '') | pd.isna(df['tariff_crawled']) | (df['tariff_crawled'] == 'Error'))).any():
        filtered_df = df.loc[(df['country'] == country) & (df['selectable'].notna()) & ((df['tariff_crawled'] == '') | pd.isna(df['tariff_crawled']) | (df['tariff_crawled'] == 'Error'))]
        print(f'row for craw: {len(filtered_df)}')
        for i in filtered_df.index:
            # if df.loc[i, 'country'] == country and (df.loc[700, 'selectable'] == True) and (df.loc[i, 'tariff_crawled'] == '' or pd.isna(df.loc[i, 'tariff_crawled']) or df.loc[i, 'tariff_crawled'] == 'Error'):
                code = df.loc[i, 'code']
                country = df.loc[i, 'country']
                attempts = 0
                success = False
                error_occurred = False  # Biến để kiểm soát lỗi

                while attempts < 5 and not success:
                    try:
                        print(f"Fetching index: {i} code: {code}... (Attempt {attempts}/5)")
                        
                        response = fetch_tariff_info_with_retry(code,country)
                        if  response and response.status_code == 200:
                            measures = response.json()[0]['measures']
                            measures_df = []
                            for measure in measures:
                                measure_str = {key: str(value) for key, value in measure.items()}
                                measure_str['hs_info_code'] = code
                                measure_str['country'] = country
                                with open('tariff_info.txt', "a", encoding='utf-8') as f:
                                    f.write(json.dumps(measure_str, ensure_ascii=False) + "\n")
                                    
                            df.loc[i, 'tariff_crawled'] = 'Done'
                            df.to_csv('hs_info - Copy.csv',index = False)

                            print(f"✅ Done fetching index: {i} code: {code}")
                            success = True
                        else:
                            raise Exception(f"Failed request, status code: {response.status_code}")
                    
                    except Exception as e:
                        attempts += 1
                        error_message = str(e)
                        print(f"❌ Error fetching index: {i} code: {code}... (Attempt {attempts}/5) - {e}")
                        if attempts == 5: 
                            error_occurred = True

                if error_occurred: 
                    df.loc[i, 'tariff_crawled'] = 'Error'
                    df.loc[i, 'tariff_crawled_error_message'] = error_message
                    df.to_csv('hs_info - Copy.csv',index = False)
                    print("⏳ Encountered multiple errors. Sleeping for 1 minutes...")
                    
                    # Mở trình duyệt để nhập captcha
                    print("🔓 Mở trình duyệt để nhập captcha...")
                    driver = webdriver.Chrome()
                    driver.get("https://trade.ec.europa.eu/access-to-markets/en/search?product=0101&origin=VN&destination=AT")  # Thay bằng URL cần mở
                    print("⌨️ Vui lòng nhập captcha, sau đó đóng trình duyệt.")

                    # Đo thời gian bắt đầu
                    start_captcha_time = time.time()
                    wait_time = 600  # 10 phút (600 giây)

                    # Chờ người dùng tự đóng trình duyệt, nếu quá 10 phút thì tự động đóng
                    while True:
                        try:
                            if time.time() - start_captcha_time > wait_time:
                                print("⏳ Quá 10 phút, tự động đóng trình duyệt.")
                                driver.quit()
                                break  # Thoát khỏi vòng lặp

                            # Kiểm tra trình duyệt còn mở không
                            driver.title  
                            
                            # Kiểm tra trang có bị treo không
                            if driver.execute_script("return document.readyState") != "complete":
                                print("⚠️ Trang web có thể bị treo, chờ thêm...")
                            
                        except WebDriverException:
                            print("✅ Trình duyệt đã được đóng.")
                            break  # Nếu trình duyệt bị đóng, thoát vòng lặp
                        
                        time.sleep(5)  # Tránh vòng lặp chạy quá nhanh, gây tải CPU
                    
                    time.sleep(10)  # Đợi 10 giây trước khi tiếp tục
                    break
            


row for craw: 16147
Fetching index: 65179 code: 0501... (Attempt 0/5)


C:\Users\nguye\AppData\Local\Temp\ipykernel_40872\3089074575.py:28: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Done' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[i, 'tariff_crawled'] = 'Done'


✅ Done fetching index: 65179 code: 0501
Fetching index: 65186 code: 0510... (Attempt 0/5)
✅ Done fetching index: 65186 code: 0510
Fetching index: 65219 code: 0814... (Attempt 0/5)
✅ Done fetching index: 65219 code: 0814
Fetching index: 65222 code: 0903... (Attempt 0/5)
✅ Done fetching index: 65222 code: 0903
Fetching index: 65246 code: 1109... (Attempt 0/5)
✅ Done fetching index: 65246 code: 1109
Fetching index: 65249 code: 1203... (Attempt 0/5)
✅ Done fetching index: 65249 code: 1203
Fetching index: 65270 code: 1506... (Attempt 0/5)


KeyboardInterrupt: 